In [2]:
import csv
from sklearn import preprocessing
import numpy as np

In [3]:
shareCount = 50
shareMap= {}
for i in range(shareCount):
    shareMap["Share"+str(i+1)] = i+1
#print (shareMap)

blank =""

In [4]:
def convertDateToFeature(date):
    #this function will be used to convert date into a numerical feature
    from datetime import datetime
    from dateutil.parser import parse
    import pandas as pd
    startdate = '23-Dec-15'
    dt = datetime.strptime( date, '%d-%b-%y' )
    sdt =datetime.strptime( startdate, '%d-%b-%y' )
    diff = dt-sdt    
    return diff.days
    

In [5]:
def transformDataShares(trainingFile,features,testsize):
    #this function will be used to load data from csv file and use preprocess it
    #Variable to hold list of list of input parameters
    transformData =[]
    
    with open(trainingFile,'rt') as csvfile:
        linereader= csv.reader(csvfile,delimiter=",",quotechar = "\"")
        lineNum =1
        for row in linereader:
            #print(row)
            #save headers in the data
            if lineNum == 1:
                header = row
            else:
                #save data in a list
                allfeatures = list(map(lambda x:shareMap[x] if row.index(x) == 0
                                 else convertDateToFeature(x) if row.index(x) == 1 else x, row))
                #1st column is share string and 2nd column is date
                #ignore all empty values
                featureVector = [allfeatures[header.index(feature)] for feature in features]
                
                #if blank not in featureVector:
                transformData.append(featureVector)
                
            lineNum += 1
    #Split data into trainng and test
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split

    train, test = train_test_split(transformData, test_size = testsize)
    train = np.array(train)
    test = np.array(test)
    #filter X and y
    label_open_train = train[:,3]
    label_open_test = test[:,3]
    label_closed_train = train[:,7]
    label_closed_test = test[:,7]
    train_open = np.delete(train, np.s_[3:4], axis=1) 
    train_open = np.delete(train_open, np.s_[6:7], axis=1) 
    train_close = np.delete(train, np.s_[7:8], axis=1) 
    #train = np.delete(train, np.s_[6:7], axis=1) 
    test_open = np.delete(test, np.s_[3:4], axis=1) 
    test_open = np.delete(test_open, np.s_[6:7], axis=1) 
    test_close = np.delete(test, np.s_[6:7], axis=1) 
    
    return transformData,train,test,train_open,train_close, test_open,test_close,label_open_train,label_open_test,label_closed_train,label_closed_test

In [6]:
filterCols = ['Share Names','Date','Prev Close','Open Price','High Price','Low Price',
            'Last Price','Close Price','Average Price','Total Traded Quantity','Turnover in Lacs',
            'Deliverable Qty','% Dly Qt to Traded Qty']
features = ['Share Names','Date','Prev Close','High Price','Low Price',
            'Last Price','Average Price','Total Traded Quantity','Turnover in Lacs',
            'Deliverable Qty','% Dly Qt to Traded Qty']
predict_y1 =['Open Price']
predict_y2 =['Close Price']

trainingFile = r'C:\Users\soursinha\Documents\Sourabh\Analytics Summit\Hackathon\HackathonRound1.csv'

transformData,train,test,train_open,train_close,test_open,test_close,label_open_train,label_open_test,label_closed_train,label_closed_test= transformDataShares(trainingFile,filterCols,0.25)

In [7]:
print(train[0], train_open[0],label_open_train[0],train_close[0],label_closed_train[0])

['49' '391' '162.65' '163.45' '163.75' '162.2' '162.55' '162.65' '162.79'
 '278579' '45349686.7' '175120' '62.86'] ['49' '391' '162.65' '163.75' '162.2' '162.55' '162.79' '278579'
 '45349686.7' '175120' '62.86'] 163.45 ['49' '391' '162.65' '163.45' '163.75' '162.2' '162.55' '162.79' '278579'
 '45349686.7' '175120' '62.86'] 162.65


In [8]:
print(test[0], test_open[0],label_open_test[0],test_close[0],label_closed_test[0])

['6' '273' '319.2' '320.4' '323' '318.8' '322' '321.85' '320.91' '1663305'
 '533772975.1' '645016' '38.78'] ['6' '273' '319.2' '323' '318.8' '322' '320.91' '1663305' '533772975.1'
 '645016' '38.78'] 320.4 ['6' '273' '319.2' '320.4' '323' '318.8' '321.85' '320.91' '1663305'
 '533772975.1' '645016' '38.78'] 321.85


In [9]:
print(len(transformData),len(train_open),len(label_open_train),len(test_open),len(label_closed_test))

#test1

23605 17703 17703 5902 5902


In [10]:
def preprocessData(inputData,ppType=None):
    if ppType == 'norm':
        return np.array(preprocessing.normalize(inputData, norm='l2')).astype(np.float)
    elif ppType == 'std':
        return preprocessing.scale(inputData)
    elif ppType == 'minmax':
        min_max_scaler = preprocessing.MinMaxScaler()
        return min_max_scaler.fit_transform(inputData)
    elif ppType == 'abs':
        max_abs_scaler = preprocessing.MaxAbsScaler()
        return max_abs_scaler.fit_transform(inputData)
    else:
        return inputData
    

In [11]:
#Preprocess Data
train_open_pp = preprocessData(train_open).astype(np.float64)
train_close_pp = preprocessData(train_close).astype(np.float64)

test_open_pp = preprocessData(test_open).astype(np.float64)
test_close_pp = preprocessData(test_close).astype(np.float64)

train_open_pp[0]
label_open_train = label_open_train.astype(np.float64)
label_open_test = label_open_test.astype(np.float64)

label_close_train = label_closed_train.astype(np.float64)
label_close_test = label_closed_test.astype(np.float64)


In [12]:
print(train_open_pp[0],label_open_train[0],label_closed_train[0])

[  4.90000000e+01   3.91000000e+02   1.62650000e+02   1.63750000e+02
   1.62200000e+02   1.62550000e+02   1.62790000e+02   2.78579000e+05
   4.53496867e+07   1.75120000e+05   6.28600000e+01] 163.45 162.65


In [13]:
#Test 1: Using linear regression
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import ARDRegression, LinearRegression

#label_open_train
ols = LinearRegression(normalize=False)
ols1 = LinearRegression()
ols.fit(train_open_pp, label_open_train)
ols1.fit(train_close_pp, label_close_train)

#Rounding off 
y_open_prediction = np.around( ols.predict(test_open_pp),decimals =4)
y_close_prediction = np.around(ols1.predict(test_close_pp),decimals =4)
label_open_test = np.around(label_open_test,decimals =4)
label_close_test = np.around(label_close_test,decimals =4)

mape_open = mean_absolute_error(label_open_test,y_open_prediction)
mape_close = mean_absolute_error(label_close_test,y_close_prediction)

#mape = mean_absolute_percentage_error(ols.predict(test_open_pp),label_open_test)
#Mean Square Error 
error_open = np.mean((y_open_prediction-label_open_test)**2)
error_close = np.mean((y_close_prediction-label_close_test)**2)
print(error_open, error_close, mape_open,mape_close)
#error without preprocessing

122.214074168 7.02353960757 5.4899601491 1.38191824805


In [14]:
y_open_prediction

array([ 320.0572,   77.092 ,   64.6649, ...,   95.8239,  579.0131,  227.75  ])

In [15]:
label_open_test

array([ 320.4 ,   77.5 ,   63.  , ...,   95.5 ,  577.5 ,  227.75])

In [16]:
y_close_prediction

array([ 321.2229,   75.8613,   63.7429, ...,   96.2372,  578.29  ,  227.258 ])

In [17]:
label_closed_test

array(['321.85', '76.15', '63.9', ..., '96.4', '577.75', '227.3'], 
      dtype='<U11')

In [18]:
np.around(y_close_prediction,decimals =2)

array([ 321.22,   75.86,   63.74, ...,   96.24,  578.29,  227.26])

In [19]:
np.around(label_close_test,decimals =2)

array([ 321.85,   76.15,   63.9 , ...,   96.4 ,  577.75,  227.3 ])

In [20]:
mean_absolute_error(label_open_test,y_open_prediction)

5.4899601491019991

In [21]:
#Using SVR
from sklearn.svm import SVR
clf = SVR(C=1e3, epsilon=0.2,kernel='rbf',gamma=0.1)
clf.fit(train_open_pp, label_open_train)
clf1 = SVR(C=1e3, epsilon=0.02)
clf1.fit(train_close_pp, label_close_train)

#Rounding off 
svr_y_open_prediction = np.around(clf.predict(test_open_pp),decimals =4)
svr_y_close_prediction = np.around(clf1.predict(test_close_pp),decimals =4)






In [24]:
mape_open_svr = mean_absolute_error(label_open_test,svr_y_open_prediction)
mape_close_svr = mean_absolute_error(label_close_test,svr_y_close_prediction)
print(mape_open_svr,mape_close_svr)
print(svr_y_open_prediction,'\n',label_open_test)
print(svr_y_close_prediction,'\n',label_close_test)

493.695857472 493.630675398
[ 640.7448  640.7448  640.7448 ...,  640.7448  640.7448  640.7448] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
[ 640.0907  640.0907  640.0907 ...,  640.0907  640.0907  640.0907] 
 [ 321.85   76.15   63.9  ...,   96.4   577.75  227.3 ]


In [25]:
test[0]

array(['6', '273', '319.2', '320.4', '323', '318.8', '322', '321.85',
       '320.91', '1663305', '533772975.1', '645016', '38.78'], 
      dtype='<U11')

In [70]:
###### Using BayesianRidge regressor
from sklearn.linear_model import BayesianRidge
clf3 = BayesianRidge(compute_score=False, tol=0.001, alpha_1=2e-07, alpha_2=2e-06, lambda_1=1e-06, lambda_2=1e-06)
clf3.fit(train_open_pp, label_open_train)

br_y_open_prediction = np.around(clf3.predict(test_open_pp),decimals =4)
br_y_open_prediction = clf3.predict(test_open_pp)

mape_open_br = mean_absolute_error(label_open_test,br_y_open_prediction)
print(mape_open_br)
print(br_y_open_prediction,'\n',label_open_test)

5.49021274376
[ 320.05628902   77.09117829   64.66513851 ...,   95.82298692  579.01405373
  227.7498929 ] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]


In [71]:
#Hubber Regressor
from sklearn.linear_model import HuberRegressor, Ridge
epsilon_values = [1.35, 1.5, 1.75, 1.9,1.1,1.2]
for k, epsilon in enumerate(epsilon_values):
    huber = HuberRegressor(fit_intercept=True, alpha=0.01, max_iter=10000,
                           epsilon=epsilon)
    huber.fit(train_open_pp, label_open_train)
    huber_y_open_prediction = np.around(huber.predict(test_open_pp),decimals =4)
    mape_open_huber = mean_absolute_error(label_open_test,huber_y_open_prediction)
    print(mape_open_huber)
    print(huber_y_open_prediction,'\n',label_open_test)
    


640.550065046
[ 145.9036    6.7234    7.1375 ...,    2.8789   34.7432  705.0954] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
640.549346747
[ 145.8957    6.7302    7.1431 ...,    2.888    34.7426  705.104 ] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
640.54933426
[ 145.8975    6.7309    7.1438 ...,    2.8878   34.7436  705.113 ] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
640.549174212
[ 145.8897    6.7324    7.1446 ...,    2.8922   34.741   705.0944] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
640.549663419
[ 145.9033    6.7237    7.1381 ...,    2.8772   34.7447  705.0882] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
640.549659471
[ 145.8999    6.7236    7.1382 ...,    2.8759   34.7445  705.0681] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]


In [72]:
# Fit a ridge regressor to compare it 
ridge = Ridge(fit_intercept=True, alpha=0.0, random_state=0, normalize=True)
ridge.fit(train_open_pp, label_open_train)
ridge_y_open_prediction = np.around(ridge.predict(test_open_pp),decimals =4)
mape_open_ridge = mean_absolute_error(label_open_test,ridge_y_open_prediction)
print(mape_open_ridge)
print(ridge_y_open_prediction,'\n',label_open_test)

5.4899601491
[ 320.0572   77.092    64.6649 ...,   95.8239  579.0131  227.75  ] 
 [ 320.4    77.5    63.   ...,   95.5   577.5   227.75]
